In [1]:
import streamlit as st
import chromadb
import json
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch


c:\Users\Govind Barbade\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data


In [17]:
def split_into_sentences(content):
    sentences = [sentence.strip() for sentence in content.split('.') if sentence.strip()]
    return sentences

In [18]:
json_file = 'data.json'
data = load_json(json_file)

In [19]:

sentences = []
metadata = []
for entry in data:
    content = entry['content']
    url = entry['url']
    entry_sentences = split_into_sentences(content)
    sentences.extend(entry_sentences)
    metadata.extend([url] * len(entry_sentences))

document_dictionary = {str(i+1): sentence for i, sentence in enumerate(sentences)}
print(len(document_dictionary))

2442


In [20]:
client = chromadb.Client()
collection_name = 'Nvidia_Docs'
# client.create_collection(name=collection_name)

collection = client.get_or_create_collection(name=collection_name)


In [21]:

# cached = st.cache(allow_output_mutation=True)
collection.add(
    documents=[x for x in list(document_dictionary.values())],
    metadatas=[{"source": metadata[i]} for i in range(len(metadata))],
    ids=[str(x) for x in document_dictionary]
)
doc_count = collection.count()
print(f"Number of documents in the collection after addition: {doc_count}")

In [ ]:
device = "cuda"
model_name = ("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

2024-07-29 13:45:18.794 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [ ]:
chat_container = st.container()
with chat_container:
    for chat in st.session_state.chat_history:
        user_query, bot_response = chat['user'], chat['bot']
        st.markdown(f"""
        <div style='display: flex; justify-content: flex-end;'>
            <div style='border: 1px solid #004d00; padding: 10px; border-radius: 10px; margin: 5px; background-color: #004d00; color: white;'>
                <b>You:</b> {user_query}
            </div>
        </div>
        <div style='display: flex; justify-content: flex-start;'>
            <div style='border: 1px solid #001a00; padding: 10px; border-radius: 10px; margin: 5px; background-color: #001a00; color: white;'>
                <b>Bot:</b> {bot_response}
            </div>
        </div>
        """, unsafe_allow_html=True)

In [ ]:
responses = [
 "Sorry, I couldn't find an answer to your question.",
 "Unfortunately, I couldn't locate an answer in the provided context.",
 "Apologies, I couldn't find relevant information to answer your question.",
 "I'm sorry, I couldn't find a suitable response in the given context.",
 "Regrettably, I couldn't extract an answer from the provided information.",
 "I apologize, I couldn't locate pertinent details to answer your question.",
 "Sorry, the question seems out of scope for the available information.",
 "Unfortunately, I couldn't determine a valid response given the context.",
 "I'm sorry, the question doesn't appear to match the available data.",
 "Apologies, I couldn't find a matching answer based on the provided context."]



In [ ]:
import random

question = input("Enter your question: ")
if question:
    results = collection.query(
        query_texts=[question],
        n_results=13
    )
    retrieved_docs = " ".join([doc for sublist in results['documents'] for doc in sublist])
    print("Response Form Database:", retrieved_docs)
    
    inputs = tokenizer(question, retrieved_docs, return_tensors="pt").to(device)

    outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    
    answer = tokenizer.decode(inputs['input_ids'][0][answer_start:answer_end], skip_special_tokens=True)
    answer = answer.replace(question, '').strip()
    if len(answer) == 0:
        answer = (responses[random.randint(0, len(responses)-1)])
    print(f"Question: {question}")
    print(f"Answer: {answer}")

Response Form Database: The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit The intent is to provide guidelines for obtaining the best performance from NVIDIA GPUs using the CUDA Toolkit 5 Update 1  Develop, Optimize and Deploy GPU-Accelerated Apps The NVIDIA® CUDA® Toolkit

In [ ]:
# What is the purpose of the NVIDIA CUDA Toolkit?
# What does the CUDA Toolkit End User License Agreement (EULA) cover?